In [1]:
using RCall
using DataFrames, DataFramesMeta
using CSV
using DataVoyager
using AMLPipelineBase
using AutoMLPipeline
using Statistics

using Distributed
nprocs() == 1 && addprocs(;exeflags="--project")
@everywhere begin
    using AutoMLPipeline
    using DataFrames
    using Statistics
end

ENV["LINES"] = 10
ENV["COLUMNS"]=10000;

┌ Info: Precompiling AutoMLPipeline [08437348-eef5-4817-bc1b-d4e9459680d6]
└ @ Base loading.jl:1278


### Load Covid Data

source: https://covid.ourworldindata.org/data/owid-covid-data.csv

In [2]:
function getdata()
   df = CSV.read("../data/owid-covid-data.csv",DataFrame);
   @rput df
   R"""
      library(tidyverse)
      library(mice)

      sdf = df %>% group_by(location) %>% 
      summarise(
        mnewcases = mean(new_cases,na.rm=T),
        mnewdeaths = mean(new_deaths,na.rm=T),
        mpopulation = mean(population,na.rm=T),
        mcardio = mean(cardiovasc_death_rate,na.rm=T),
        mdiab = mean(diabetes_prevalence,na.rm=T),
        mhandwash = mean(handwashing_facilities,na.rm=T),
        mhbed = mean(hospital_beds_per_thousand,na.rm=T),
        mpatient = mean(hosp_patients,na.rm=T),
        madmission = mean(weekly_hosp_admissions,na.rm=T),
        mhdi = mean(human_development_index,na.rm=T),
        mle = mean(life_expectancy,na.rm=T)
      ) 
      imputed=mice(sdf,meth='sample',printFlag=F)
      complete(imputed)
   """ |> rcopy
end;

In [3]:
dfimp = getdata()
X = dfimp[:,Not([:location,:mnewdeaths])]
Y = dfimp.mnewdeaths;

┌ Warning: RCall.jl: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
│ ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
│ ✔ tibble  3.0.4     ✔ dplyr   1.0.2
│ ✔ tidyr   1.1.2     ✔ stringr 1.4.0
│ ✔ readr   1.4.0     ✔ forcats 0.5.0
│ ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
│ ✖ dplyr::filter() masks stats::filter()
│ ✖ dplyr::lag()    masks stats::lag()
└ @ RCall /Users/ppalmes/.julia/packages/RCall/eRsxl/src/io.jl:160
┌ Warning: RCall.jl: 
│ Attaching package: ‘mice’
│ 
│ The following object is masked from ‘package:stats’:
│ 
│     filter
│ 
│ The following objects are masked from ‘package:base’:
│ 
│     cbind, rbind
│ 
└ @ RCall /Users/ppalmes/.julia/packages/RCall/eRsxl/src/io.jl:160
┌ Warning: RCall.jl: `summarise()` ungrouping output (override with `.groups` argument)
└ @ RCall /Users/ppalmes/.julia/packages/RCall/eRsxl/src/io.jl:160
┌ Warning: RCall.jl: Warning: Number of logged events: 1
└ @ RCall /Users/ppalmes/.ju

### Baseline performance using Random Forest

In [20]:
R"""
library(caret)

fitControl <- trainControl(method = "repeatedcv",number = 5)
rfcaret <- train($X,$Y, method = "rf",trControl = fitControl)
rfcaret
"""

RObject{VecSxp}
Random Forest 

190 samples
 10 predictor

No pre-processing
Resampling: Cross-Validated (5 fold, repeated 1 times) 
Summary of sample sizes: 151, 152, 152, 152, 153 
Resampling results across tuning parameters:

  mtry  RMSE      Rsquared   MAE     
   2    54.65968  0.6606754  22.15499
   6    45.43660  0.7791667  15.80200
  10    44.47021  0.8219964  14.13773

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was mtry = 10.


#### NOTE: Best RMSE is around 44

### Use AutoML to find better RMSE by searching optimal pipelines

In [5]:
using AutoMLPipeline

#### Decomposition
pca = SKPreprocessor("PCA")
fa  = SKPreprocessor("FactorAnalysis")
ica = SKPreprocessor("FastICA")

#### Scaler 
rb   = SKPreprocessor("RobustScaler")
pt   = SKPreprocessor("PowerTransformer")
norm = SKPreprocessor("Normalizer")
mx   = SKPreprocessor("MinMaxScaler")
mstd    = SKPreprocessor("StandardScaler")

#### categorical preprocessing
ohe = OneHotEncoder()
noop = Identity(Dict(:name=>"Noop"))

#### Column selector
catf = CatFeatureSelector()
numf = NumFeatureSelector()

#### Learners
sk_rf       = SKLearner("RandomForestRegressor")
sk_gb       = SKLearner("GradientBoostingRegressor")
sk_svr      = SKLearner("SVR")
sk_gp       = SKLearner("GaussianProcessRegressor")
sk_ard      = SKLearner("ARDRegression")
jl_rf       = RandomForest()
jl_vote     = VoteEnsemble()
jl_stack    = StackEnsemble()
jl_best     = BestLearner();

In [6]:
plsvc = @pipeline  numf |> rb |> sk_svr
pred = fit_transform!(plsvc,X,Y)

190-element Array{Float64,1}:
 1.8818731662116281
 1.3110213771957344
 3.02758398489245
 ⋮
 0.8148068720569519
 0.8149204723981107

In [22]:
@everywhere rmse(x,y) = sqrt(mean((x .- y).^2))

plsvc = @pipeline ((numf |> rb |> ica )) |> sk_rf
crossvalidate(plsvc,X,Y,rmse)

fold: 1, 23.75377291811001
fold: 2, 118.3160119227099
fold: 3, 31.333725620472336
fold: 4, 25.83782646779016
fold: 5, 89.29041825244632
fold: 6, 7.989249499667266
fold: 7, 73.58475299973867
fold: 8, 57.94725987307732
fold: 9, 13.984975191955701
fold: 10, 35.82085383553789
errors: 0


(mean = 47.78588465815056, std = 36.002051049719974, folds = 10, errors = 0)

### Parallel algo to search optimal combinations of preprocessing elements and learner

In [23]:
function prpsearch(X,Y)
    learners = [sk_rf,sk_gb,sk_svr,sk_ard,sk_gp,jl_rf];
    scalers = [rb,pt,norm,mx,mstd,noop];
    extractors = [ica,noop];
    dftable = @sync @distributed (vcat) for lr in learners
    @distributed (vcat) for sc in scalers
     @distributed (vcat) for xt  in extractors
          pipe  = @pipeline (numf |> sc |> xt)  |> lr
          scn   = sc.name[1:end - 4]; xtn = xt.name[1:end - 4]; lrn = lr.name[1:end - 4]
          pname = "$scn |> $xtn |> $lrn"
          ptime = @elapsed begin
             mean, sd, kfold, _ = crossvalidate(pipe, X, Y, rmse, 5,true)
          end
          DataFrame(pipeline=pname, mean=mean, sd=sd, time=ptime, folds=kfold)
        end
     end
    end
    sort!(dftable, :mean, rev=false);
    dftable
end

prpsearch (generic function with 1 method)

In [9]:
ENV["LINES"]=1000

runtime = @elapsed begin
    dfres = prpsearch(X,Y)
end;
serialtime = dfres.time |> sum;

(serialtime = "$(round(serialtime / 60.0)) minutes", 
    paralleltime = "$(round(runtime)) seconds")

      From worker 2:	fold: 1, 142.25146403445717
      From worker 5:	fold: 1, 32.50584127021188
      From worker 6:	fold: 1, 1213.9015660141276
      From worker 4:	fold: 1, 75.2578182491557
      From worker 6:	fold: 2, 1529.8005363946424
      From worker 5:	fold: 2, 9.587368395916121
      From worker 5:	fold: 3, 79.44117771857492
      From worker 2:	fold: 2, 47.38469350623029
      From worker 3:	fold: 1, 8.254081313172394
      From worker 4:	fold: 2, 31.816555976518863
      From worker 6:	fold: 3, 813.6011519195193
      From worker 6:	fold: 4, 61.49262552957701
      From worker 6:	fold: 5, 86.75153121570264
      From worker 6:	errors: 0
      From worker 3:	fold: 2, 88.41329767031164
      From worker 5:	fold: 4, 47.61135746983782
      From worker 3:	fold: 3, 53.08320397635213
      From worker 6:	fold: 1, 53.179985236752195
      From worker 3:	fold: 4, 76.40442430171119
      From worker 6:	fold: 2, 11.707490479899207
      From worker 6:	fold: 3, 32.91475619435752
    

      From worker 3:	fold: 1, 78.08826307865891
      From worker 3:	fold: 2, 30.071325157521567
      From worker 6:	fold: 4, 40.73323731669982
      From worker 3:	fold: 3, 88.75550477014359
      From worker 3:	fold: 4, 148.41296073711152
      From worker 7:	fold: 4, 30.833130737645774
      From worker 3:	fold: 5, 81.67163346464066
      From worker 3:	errors: 0
      From worker 6:	fold: 5, 67.05832382114728
      From worker 6:	errors: 0
      From worker 6:	fold: 2, 25.565461107591762
      From worker 6:	fold: 3, 154.6884447046042
      From worker 6:	fold: 4, 38.97042291762992
      From worker 6:	fold: 5, 52.51039868017175
      From worker 6:	errors: 0
      From worker 7:	fold: 5, 52.12694615902805
      From worker 7:	errors: 0
      From worker 7:	fold: 3, 10.095278092421879
      From worker 7:	fold: 4, 130.16387696136061
      From worker 7:	fold: 5, 20.122448774585664
      From worker 7:	errors: 0
      From worker 2:	fold: 4, 42.15702056274721
      From worker 2:	f

      From worker 2:	fold: 2, 91.64428828725055
      From worker 2:	fold: 3, 55.29406800697017
      From worker 2:	fold: 4, 164.5210307326601
      From worker 2:	fold: 5, 169.74817258644512
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 29.77487837893025
      From worker 2:	fold: 2, 53.644127105380164
      From worker 2:	fold: 3, 128.83530742601315
      From worker 2:	fold: 4, 16.112152068037883
      From worker 2:	fold: 5, 17.910048511959673
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 34.98841607048003
      From worker 2:	fold: 2, 89.26403978864603
      From worker 2:	fold: 3, 163.13364123568869
      From worker 2:	fold: 4, 43.80202613906051
      From worker 2:	fold: 5, 72.23983663753434
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 2161.702498402926
      From worker 2:	fold: 2, 30051.754401948878
      From worker 2:	fold: 3, 68.0561543861318
      From worker 2:	fold: 4, 189.95527942119665
      From worker 2:	fold: 5, 168.60

(serialtime = "9.0 minutes", paralleltime = "39.0 seconds")

In [10]:
dfres

,pipeline,mean,sd,time,folds
,String,Float64,Float64,Float64,Int64
1,Noop |> Noop |> ARDRegression,24.2595,19.3394,1.8202,5
2,StandardScaler |> FastICA |> ARDRegression,32.0012,15.4195,4.75838,5
3,MinMaxScaler |> FastICA |> ARDRegression,32.611,16.4626,5.84648,5
4,StandardScaler |> Noop |> ARDRegression,34.3646,16.6915,10.7387,5
5,RobustScaler |> Noop |> ARDRegression,36.2977,31.1167,9.52637,5
6,PowerTransformer |> Noop |> RandomForestRegressor,39.3629,34.3177,8.4027,5
7,MinMaxScaler |> Noop |> GradientBoostingRegressor,39.762,30.408,11.7218,5
8,RobustScaler |> FastICA |> ARDRegression,40.6137,25.6402,8.8369,5
9,MinMaxScaler |> Noop |> RandomForestRegressor,41.0478,28.0872,12.0898,5


### Extend the pipeline with R ML implementations

In [31]:
module RMachine

using RCall
using DataFrames
using Random

using AMLPipelineBase
using AMLPipelineBase.AbsTypes
using AMLPipelineBase.Utils

import AMLPipelineBase.AbsTypes: fit!, transform!

export CRTLearner,fit!,transform!,fit_transform!
export caretrun, crossvalidate

mutable struct CRTLearner <: Learner
   name
   model
   function CRTLearner(args::Dict{Symbol,<:Any} = Dict())
      fitControl="trainControl(method = 'none')"
      default_args = Dict{Symbol,Any}(
                      :name => "R_learner",
                      :learner => "rf",
                      :fitControl => fitControl,
                      :impl_args => Dict{Symbol,Any}()
                     )
      cargs = nested_dict_merge(default_args, args)
      cargs[:name] = cargs[:name]*"_"*randstring(3)
      rl = cargs[:learner]
      new(cargs[:name],cargs)
   end
end

function fit!(crt::CRTLearner,x::DataFrame,y::Vector)
   rmodel = rcall(:train,x,y,method=crt.model[:learner],trControl = reval(crt.model[:fitControl]))
   crt.model[:rmodel] = rmodel
end

function transform!(crt::CRTLearner,x::DataFrame)
   res = rcall(:predict,crt.model[:rmodel],x)
   return rcopy(res) |> Array
end


end

Main.RMachine

In [33]:
module RTest
using Test
using DataFrames
using RDatasets

using ..RMachine

function test_r()
   rf = CRTLearner(Dict(:learner=>"rf"))
   gbm = CRTLearner(Dict(:learner=>"gbm"))
   iris=dataset("datasets","iris")
   x=iris[:,1:4]
   y=iris[:,5] |> Array{String}
   @test (fit_transform!(rf,x,y) .== y ) |> sum == 150
   @test crossvalidate(rf,x,y,"accuracy_score",3,false).mean > 0.80
end
@testset "rmachine tests" begin
   test_r()
end
end

Test Summary:  | Pass  Total
rmachine tests |    2      2


Main.RTest

In [27]:
@everywhere using AMLPipelineR

In [34]:
r_rf = CRTLearner(Dict(:name=>"RF_fromR",:learner=>"rf"));

In [35]:
function prpsearch(X,Y)
    learners = [sk_rf,sk_gb,sk_svr,sk_ard,sk_gp,jl_rf,r_rf];
    scalers = [rb,pt,norm,mx,mstd,noop];
    extractors = [ica,noop];
    dftable = @sync @distributed (vcat) for lr in learners
    @distributed (vcat) for sc in scalers
     @distributed (vcat) for xt  in extractors
          pipe  = @pipeline (numf |> sc |> xt)  |> lr
          scn   = sc.name[1:end - 4]; xtn = xt.name[1:end - 4]; lrn = lr.name[1:end - 4]
          pname = "$scn |> $xtn |> $lrn"
          ptime = @elapsed begin
             mean, sd, kfold, _ = crossvalidate(pipe, X, Y, rmse, 5,true)
          end
          DataFrame(pipeline=pname, mean=mean, sd=sd, time=ptime, folds=kfold)
        end
     end
    end
    sort!(dftable, :mean, rev=false);
    dftable
end

runtime = @elapsed begin
    dfres = prpsearch(X,Y)
end;
serialtime = dfres.time |> sum;

(serialtime = "$(round(serialtime / 60.0)) minutes", 
    paralleltime = "$(round(runtime)) seconds")

      From worker 8:	fold: 1, 33.87167649949282
      From worker 7:	fold: 1, 67.41005090137035
      From worker 7:	fold: 1, 64.06786583131652
      From worker 7:	fold: 2, 38.6355379532262
      From worker 7:	fold: 3, 25.085354329926577
      From worker 7:	fold: 4, 66.50374543466512
      From worker 7:	fold: 5, 14.483361972665591
      From worker 7:	errors: 0
      From worker 7:	fold: 1, 58.12911493975861
      From worker 8:	fold: 2, 93.64898214369772
      From worker 5:	fold: 1, 18.07299471250063
      From worker 3:	fold: 1, 11.253499585552964
      From worker 6:	fold: 1, 805.4605307559489
      From worker 5:	fold: 2, 58.2529503384154
      From worker 4:	fold: 1, 150.29340096569345
      From worker 7:	fold: 2, 16.403777032572815
      From worker 6:	fold: 2, 77.41815525479717
      From worker 6:	fold: 3, 278.7220174146623
      From worker 4:	fold: 2, 81.76013058452963
      From worker 5:	fold: 3, 26.23263908360398
      From worker 6:	fold: 4, 37.59812230202251
      

      From worker 4:	fold: 3, 32.70003307722844
      From worker 4:	fold: 4, 25.404398756557857
      From worker 4:	fold: 5, 52.99512789523824
      From worker 4:	errors: 0
      From worker 6:	fold: 5, 30.297574779144544
      From worker 6:	errors: 0
      From worker 6:	fold: 1, 139.70275345048273
      From worker 5:	fold: 5, 27.527487143028505
      From worker 5:	errors: 0
      From worker 5:	fold: 1, 116.55298869676494
      From worker 6:	fold: 2, 30.408801761477836
      From worker 6:	fold: 3, 40.958499437432806
      From worker 5:	fold: 2, 61.621179268748655
      From worker 6:	fold: 4, 22.65122659038507
      From worker 6:	fold: 5, 53.306052878980594
      From worker 6:	errors: 0
      From worker 5:	fold: 3, 59.14114578415045
      From worker 5:	fold: 4, 32.60949531401328
      From worker 5:	fold: 5, 21.654968961832065
      From worker 5:	errors: 0
      From worker 3:	fold: 5, 102.48342420483817
      From worker 3:	errors: 0
      From worker 3:	fold: 1, 144.5

      From worker 2:	fold: 4, 35.342715614116614
      From worker 2:	fold: 5, 93.8645578928859
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 17.89195078817202
      From worker 2:	fold: 2, 29.4049012151928
      From worker 2:	fold: 3, 21.671781734207773
      From worker 2:	fold: 4, 30.87389765845601
      From worker 2:	fold: 5, 48.93905650768662
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 45.363367462716106
      From worker 2:	fold: 2, 128.45919091238792
      From worker 2:	fold: 3, 48.757337002752905
      From worker 2:	fold: 4, 71.20347054963699
      From worker 2:	fold: 5, 20.573178385414575
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 162.97995382073427
      From worker 2:	fold: 2, 102.01900682919246
      From worker 2:	fold: 3, 10.629822195908789
      From worker 2:	fold: 4, 75.25912883816855
      From worker 2:	fold: 5, 30.83112023611581
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 10.891351182589268
    

      From worker 2:	fold: 2, 33.444015948235936
      From worker 2:	fold: 3, 46.10959369949265
      From worker 2:	fold: 4, 135.10863398592642
      From worker 2:	fold: 5, 24.861537526664048
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 128.0506497440898
      From worker 2:	fold: 1, 46.46854819784164
      From worker 2:	fold: 2, 31.02951172128375
      From worker 2:	fold: 3, 88.80601040860256
      From worker 2:	fold: 4, 43.82918110849347
      From worker 2:	fold: 5, 117.87566501345913
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 91.6521208746042
      From worker 2:	fold: 2, 31.280454417957046
      From worker 2:	fold: 3, 2.8505799881761114
      From worker 2:	fold: 4, 10.869148296598896
      From worker 2:	fold: 5, 40.75503636424105
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 113.35134408928816
      From worker 2:	fold: 2, 48.28309882103406
      From worker 2:	fold: 3, 21.518633065817248
      From worker 2:	fold: 4, 71.51

(serialtime = "16.0 minutes", paralleltime = "30.0 seconds")

In [36]:
dfres

,pipeline,mean,sd,time,folds
,String,Float64,Float64,Float64,Int64
1,StandardScaler |> Noop |> ARDRegression,27.3874,16.1732,8.38338,5
2,MinMaxScaler |> Noop |> ARDRegression,29.4218,16.0239,9.56574,5
3,RobustScaler |> Noop |> ARDRegression,29.7563,11.9945,20.9123,5
4,MinMaxScaler |> FastICA |> ARDRegression,30.5677,18.7811,4.89073,5
5,Noop |> FastICA |> ARDRegression,32.8489,12.7548,3.26554,5
6,PowerTransformer |> Noop |> RandomForestRegressor,35.4815,34.8968,23.558,5
7,Noop |> Noop |> ARDRegression,39.7447,30.9597,23.8513,5
8,PowerTransformer |> Noop |> GradientBoostingRegressor,40.3849,31.2169,9.57607,5
9,Noop |> FastICA |> GradientBoostingRegressor,41.7552,23.1393,1.13656,5
